In this notebook Longitude and latitude columns are added in order for further visualisations and processing. This proccess take a lot of time, therefore it is done in separate notebook and columns added before all further manipulation with the dataset.

In [ ]:
from geopy.geocoders import Nominatim
import pandas as pd
import time

In [ ]:
df = pd.read_parquet("../data/raw/pollution_dataset.parquet", engine="pyarrow")
df.head()

In [ ]:


# 1 — Extract unique addresses
unique_addresses = pd.DataFrame(df["Address"].unique(), columns=["Address"])
print("Unique addresses:", len(unique_addresses))

# 2 — Prepare columns
unique_addresses["Latitude"] = None
unique_addresses["Longitude"] = None

# 3 — Initialize geocoder
geolocator = Nominatim(user_agent="my_geocoder_app")

def geocode_address(address):
    try:
        location = geolocator.geocode(address)
        if location:
            return location.latitude, location.longitude
        return None, None
    except:
        return None, None

# 4 — Loop with delay
coords = []
for addr in unique_addresses["Address"]:
    lat, lon = geocode_address(addr)
    coords.append((lat, lon))
    time.sleep(1)   # Respect Nominatim rate limits

unique_addresses["Latitude"], unique_addresses["Longitude"] = zip(*coords)

# 5 — Save to parquet
unique_addresses.to_parquet("../data/preprocessed/pollution_dataset_coordinates.parquet", index=False)
print("Saved geocoded addresses to address_coordinates.parquet")